## Аггрегация разметки датасета BPS

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик.

In [1]:
import pandas as pd
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Датасет проходил разметку одним пулом из 100 сгенерированных специально объектов в формате оригинального датасета.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41554142__29-09-2023.tsv', sep='\t')
skills = pd.read_csv('workerSkills.csv', sep='|')

Разметчикам предлагалось на основании имеющейся скобочной последовательности ответить на вопрос, сбалансирована ли она или нет. 

Вход: 
- INPUT:parentheses (пример: `[ { ( [ ] ) } ]`).

Выход:
- OUTPUT:correct (0 или 1).

In [3]:
assignments.head(1)

,INPUT:parentheses,OUTPUT:correct,GOLDEN:correct,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,[ ( } ) ) ] { [ ] ( { { [ ( ] } } ) ) {,0,NaN,NaN,NaN,https://platform.toloka.ai/task/41554142/00027...,00027a10de--6515e7f4e6499645e520ca19,00027a10de--651672354dc98f156e8fc965,00027a10de--651672354dc98f156e8fc95b,ec3328bcaa866d99022b38cf8e522237,APPROVED,2023-09-29T06:44:05.190,2023-09-29T06:44:22.420,2023-09-29T06:44:22.420,0.022


Фильтруем толокеров, которые дали меньше половины корректных ответов на контрольных заданиях.

In [4]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    para = row[0]

    out = row[1]
    
    gold = row[2]

    user = row[9]

    if str(user) != "nan" and str(gold) != "nan":
        if out == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  24
Bad users: 3


3 из 24 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Теперь нужно оставить только основной пул. Контролки генерировались отдельно от самой подвыборки, поэтому их учитывать нельзя. На контрольных заданиях есть `GOLDEN:correct`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:parentheses` содержит NaN.

In [5]:
assignments_no_control = assignments[assignments['GOLDEN:correct'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:parentheses'].notnull()]

Посчитаем, сколько было затрачено на получение разметки тестовых данных без учета контрольных заданий, так как они могли проходиться неограниченное количество раз одним и тем же разметчиком.

In [6]:
def w_sum(df):
    idx = df.index.values
    vals = df.values
    summ = idx * vals
    return summ.sum()

d1 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts(normalize=True)
d2 = assignments_no_control_no_null['ASSIGNMENT:reward'].value_counts()
print(f'взвешенная цена айтема в тесте: {round(w_sum(d1), 3)}')
print(f'потрачено на разметку теста: {round(w_sum(d2), 3)}')
print(f'{round(w_sum(d2), 3)} / {round(w_sum(d1), 3)}')

взвешенная цена айтема в тесте: 0.02
потрачено на разметку теста: 10.172
10.172 / 0.02


Выделим, сколько составила средняя часовая ставка для разметки тестовой части датасета. Это будет простое среднее из следующих величин: количество заданий, которое разметчк может сделать за час на основе данного задания, помноженное на цену задания.

In [7]:
def get_hour_pay(df):
    try:
        times = pd.to_datetime(df['ASSIGNMENT:submitted']) - pd.to_datetime(df['ASSIGNMENT:started'])
    except Exception as e:
        times = []
        for i in range(len(assignments_no_control_no_null)):
            try:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:started'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:started'].apply(lambda x: x.split('T')[1]).iloc[i])
            try:
                end = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:submitted'].iloc[i])
            except Exception as e:
                start = pd.to_datetime(assignments_no_control_no_null['ASSIGNMENT:submitted'].apply(lambda x: x.split('T')[1]).iloc[i])
            delta = end - start
            times.extend([delta])
        times = pd.Series(times)
    sums = 3600 / times.apply(lambda x: x.seconds) * df['ASSIGNMENT:reward']
    return sums.mean()

get_hour_pay(assignments_no_control_no_null)

3.1774317196437925

### Сбор ответов разметчиков и голосование

Собираем ответы для каждого задания.

In [8]:
assignments.head(2)

,INPUT:parentheses,OUTPUT:correct,GOLDEN:correct,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,[ ( } ) ) ] { [ ] ( { { [ ( ] } } ) ) {,0,NaN,NaN,NaN,https://platform.toloka.ai/task/41554142/00027...,00027a10de--6515e7f4e6499645e520ca19,00027a10de--651672354dc98f156e8fc965,00027a10de--651672354dc98f156e8fc95b,ec3328bcaa866d99022b38cf8e522237,APPROVED,2023-09-29T06:44:05.190,2023-09-29T06:44:22.420,2023-09-29T06:44:22.420,0.022
1,{ ] { ],1,0.0,NaN,NaN,https://platform.toloka.ai/task/41554142/00027...,00027a10de--6515e7e063c0d47637f1e106,00027a10de--651672354dc98f156e8fc965,00027a10de--651672354dc98f156e8fc95b,ec3328bcaa866d99022b38cf8e522237,APPROVED,2023-09-29T06:44:05.190,2023-09-29T06:44:22.420,2023-09-29T06:44:22.420,0.022


In [9]:
from collections import defaultdict

text_dict = defaultdict(list)

for para, user, out in zip(
    assignments_no_control_no_null["INPUT:parentheses"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:correct"]
    ):
    if user not in bad_users:
        text_dict[(para)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

100


In [10]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 91, 4: 8, 3: 1})

Есть 9 заданий с перекрытием меньше 5. Однако в них всех есть большинство, проголосовавшее за одинаковый ответ, потому не нужно оценивать скиллы разметчиков для вычисления итоговых меток.

In [14]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    if len(lst) == 5:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most >= 3:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res
    elif len(lst) <= 4:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most > 2:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res

In [15]:
len(preds_full)

100

In [16]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['parentheses']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

Во всех 100 объектах согласованность достаточно высока.

In [17]:
preds_full_df.head(5)

,parentheses,lb
0,[ { ( [ ] ) } ],1
1,{ } [ ] [ ] ( ),1
2,[ ] [ ] [ ] [ ] [ ( ) { ( ) } ( ) ( ) ],1
3,( ] } ],0
4,[ { [ },0


В данном датафрейме собраны все задания и результаты голосования по ним большинством.

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными метками.

In [20]:
res_df = pd.read_csv('full_with_ans.tsv', sep='\t')

In [21]:
res_df = res_df.rename({'INPUT:parentheses': 'parentheses', 'target': 'lb'}, axis=1)

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [22]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['parentheses'] = res_df['parentheses'].apply(format_text)
preds_full_df['parentheses'] = preds_full_df['parentheses'].apply(format_text)

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [23]:
new = res_df.merge(preds_full_df, on='parentheses', how='left')

In [24]:
new_valid = new[new['lb_y'].notna()].copy()

In [25]:
len(new_valid)

100

In [26]:
new_valid.head(1)

,parentheses,lb_x,lb_y
0,[ ) { { [ ( ) } ) { ] [ ) ] } { } ( ( ],0,0


### Подсчет метрики

Если в правом столбце меток осталось 100 непустых строк, значит, форматирование было подчищено корректно, и ничего не потерялось.

In [27]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [28]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

1.0

`Accuracy = 1.0`

### Анализ ошибок

Начнем анализ с рассмотрения отдельных ответов разметчиков.

In [29]:
new = assignments_no_control_no_null.iloc[:, [0, 1, 9]].rename({
    'INPUT:parentheses': 'parentheses',
    'OUTPUT:correct': 'label',
    'ASSIGNMENT:worker_id': 'user'
}, axis=1).copy()

In [30]:
new = new.merge(res_df, on='parentheses', how='left')

In [31]:
(new['label'] != new['lb']).sum()

25

Есть всего 25 случаев, когда разметчик ответил неправильно на задание. Ни в одном задании неправильных ответов не было больше 2, потому голосованием большинством ошибки затерлись.

In [32]:
import numpy as np
from scipy import stats

Посмотрим на ошибки внимательнее.

In [33]:
new['error'] = (new['label'] != new['lb']).astype(int)

Возможно, они связаны с длиной задания или количеством разных уникальных скобок (от 1 до 3).

In [34]:
new['lens'] = new.parentheses.apply(lambda x: len(x.replace(' ', '')))
new['types'] = new.parentheses.apply(lambda x: int(np.ceil(len(set(x.replace(' ', ''))) / 2)))

In [35]:
pd.DataFrame({
    "Корреляция ошибок и длин последовательностей": stats.pearsonr(new['lens'], new['error'])[0],
    "Корреляция и количества разных типов скобок": stats.pearsonr(new['types'], new['error'])[0]
    }, index=['Корреляция Пирсона']).round(3).T

,Корреляция Пирсона
Корреляция ошибок и длин последовательностей,0.092
Корреляция и количества разных типов скобок,0.045


Ошибки носят случайный характер и не взаимосвязаны напрямую с длиной входной последовательности, а также с количеством разных типов скобок.

Вероятно, можно оценить сложность каждого примера через то, сколько нужно сделать шагов простейшим алгоритмом решения задачи, а также длину нерешаемого остатка для некорректных скобочных последовательностей.

In [36]:
def check(obj):
    text = obj.replace(' ', '')
    cnt = 0
    while '()' in text or '[]' in text or '{}' in text:
        if '()' in text:
            text = text.replace('()', '')
            cnt += 1
        if '[]' in text:
            text = text.replace('[]', '')
            cnt += 1
        if '{}' in text:
            text = text.replace('{}', '')
            cnt += 1
    return (int(not text), cnt, text)

In [37]:
new['path'] = new['parentheses'].apply(lambda x: check(x)[1])
new['rest'] = new['parentheses'].apply(lambda x: len(check(x)[2]))

In [38]:
pd.DataFrame({
    "Корреляция ошибок и количества простейших скобок": stats.pearsonr(new['lens'], new['error'])[0],
    "Корреляция и количества оставшихся нерешаемых скобок": stats.pearsonr(new['types'], new['error'])[0]
    }, index=['Корреляция Пирсона']).round(3).T

,Корреляция Пирсона
Корреляция ошибок и количества простейших скобок,0.092
Корреляция и количества оставшихся нерешаемых скобок,0.045


Получается, действительно, есть слабая взаимосвязь между количеством шагов для решения задачи и вероятностью ошибки на таком примере. Небольшая отрицательная корреляция для длин остатков может означать, что в таких остатках есть однозначные признаки нерешаемости, например, только одна открывающая скобка определенного вида (без закрывающей).